### Import all necessary Libraries

In [1]:
import pandas as pd
import numpy as np


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline



import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost
from sklearn.ensemble import AdaBoostClassifier

In [4]:
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import norm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from xgboost import XGBClassifier

In [5]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, accuracy_score
from sklearn import metrics

In [6]:
import time

In [7]:
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report,precision_score,recall_score,roc_curve,auc,accuracy_score,f1_score
# from imblearn.over_sampling import SMOTE
from collections import Counter

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report,precision_score,recall_score,roc_curve,auc,accuracy_score,f1_score

### Read the Dataset

In [8]:
finaldata = pd.read_csv("Selected_Breast_cancer.csv")

### Check the outliers using quantile methods

In [9]:
for i in finaldata.columns:
    Q1,Q3 = np.quantile(finaldata[i],[0.25,0.75])
    IQR = Q3-Q1
    lower_bound = Q1 - (1.5*IQR)
    if lower_bound < 0:  # as in whole dataset there is no -ve value hence we set lowerbound to 0
        lower_bound=0
    upper_bound = Q3 + (1.5*IQR)
    print(i)
    print('Lower Bound =',np.round(lower_bound,2),' Upper Bound =',np.round(upper_bound,2))
    print('min value =',finaldata[i].min(), ' max value =', finaldata[i].max())

    if finaldata[i].min() < lower_bound:
        print('negative Outliers',len(finaldata[(finaldata[i]<lower_bound)]))
    
    if finaldata[i].max() > upper_bound:
        print('positive Outliers', len(finaldata[(finaldata[i]>upper_bound)]))  
        
   
        
       
    
    print('='*50)

texture_mean
Lower Bound = 0  Upper Bound = 0.69
min value = 0.0  max value = 1.0
positive Outliers 7
concavity_mean
Lower Bound = 0  Upper Bound = 0.66
min value = 0.0  max value = 1.0
positive Outliers 18
symmetry_mean
Lower Bound = 0.03  Upper Bound = 0.71
min value = 0.0  max value = 1.0
negative Outliers 1
positive Outliers 14
texture_se
Lower Bound = 0  Upper Bound = 0.46
min value = 0.0  max value = 1.0
positive Outliers 20
area_se
Lower Bound = 0  Upper Bound = 0.15
min value = 0.0  max value = 1.0000000000000002
positive Outliers 65
fractal_dimension_se
Lower Bound = 0  Upper Bound = 0.25
min value = 0.0  max value = 1.0
positive Outliers 28
diagnosis
Lower Bound = 0  Upper Bound = 2.5
min value = 0  max value = 1


In [11]:
X= finaldata.drop('diagnosis',axis =1)
y= finaldata['diagnosis']

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((455, 6), (114, 6), (455,), (114,))

## Hyperparameter Optimization : GridSearch with Cross Validation for tuning parameters

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.ensemble import ExtraTreesClassifier
# import xgboost
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import GridSearchCV

In [ ]:
ad  = AdaBoostClassifier()

In [19]:
Algo_tuning = []
Training_tuning = []
Testing_tuning = []

In [20]:
%%time
# A parameter grid for Logistic Regression

from sklearn.model_selection import GridSearchCV
clf = LogisticRegression()

# Hyperparameter Optimization
parameters = {'penalty':['l1', 'l2'],'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
'class_weight' :[{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}],
'solver' : ['liblinear', 'saga'] }

# Run the grid search
grid_obj = GridSearchCV(clf, parameters,cv=10,scoring= 'accuracy', n_jobs= -1, verbose= 3)
    
grid_obj = grid_obj.fit(X_train, y_train)

# Set the knn to the best combination of parameters
clf = grid_obj.best_estimator_

# Train the model using the training sets 
clf.fit(X_train,y_train)
y_pred_clf_test = clf.predict(X_test)
y_pred_clf_train = clf.predict(X_train)



Fitting 10 folds for each of 128 candidates, totalling 1280 fits
Wall time: 5.67 s


In [21]:
# Calculating the accuracy
acc_clf_test =  metrics.accuracy_score(y_test, y_pred_clf_test) 
acc_clf_train =  metrics.accuracy_score(y_train, y_pred_clf_train) 

print( 'Accuracy of Logistic Regression model training : ', acc_clf_train )
print( 'Accuracy of Logistic Regression model testing : ', acc_clf_test )

roc_clf_train =metrics.roc_auc_score(y_train,y_pred_clf_train )
roc_clf_test = metrics.roc_auc_score(y_test,y_pred_clf_test )


Accuracy of Logistic Regression model training :  0.9604395604395605
Accuracy of Logistic Regression model testing :  0.9649122807017544


In [22]:
Algo_tuning.append('Logistic Regression')
Training_tuning.append(acc_clf_train)
Testing_tuning.append(acc_clf_test)

In [23]:
%%time
# Create a Support Vector Classifier
svc = SVC()

# Hyperparameter Optimization
parameters = {
  'kernel': ['linear','rbf'],
  'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], 
   'class_weight' :[{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}],
  'gamma': [0.001, 0.0001],
    'probability':[True]
             }
   


# Run the grid search
grid_obj = GridSearchCV(svc, parameters,cv=10,scoring= 'accuracy', n_jobs= -1, verbose= 3)
    
grid_obj = grid_obj.fit(X_train, y_train)

# Set the svc to the best combination of parameters
svc = grid_obj.best_estimator_


# Train the model using the training sets 
svc.fit(X_train,y_train)
y_pred_svc_test = svc.predict(X_test)
y_pred_svc_train = svc.predict(X_train)



Fitting 10 folds for each of 128 candidates, totalling 1280 fits
Wall time: 20.2 s


In [24]:
# Calculating the accuracy
acc_svc_test =  metrics.accuracy_score(y_test, y_pred_svc_test)
acc_svc_train = metrics.accuracy_score(y_train, y_pred_svc_train) 
print( 'Accuracy of SVC model training : ', acc_svc_train )
print( 'Accuracy of SVC model testing : ', acc_svc_test )

roc_svc_train =metrics.roc_auc_score(y_train,y_pred_svc_train )
roc_svc_test = metrics.roc_auc_score(y_test,y_pred_svc_test )


Accuracy of SVC model training :  0.9538461538461539
Accuracy of SVC model testing :  0.9649122807017544


In [25]:
Algo_tuning.append('SVC')
Training_tuning.append(acc_svc_train)
Testing_tuning.append(acc_svc_test)

In [26]:
%%time
# K - Nearest Neighbors

# Create a KNN Classifier
knn = KNeighborsClassifier()

# Hyperparameter Optimization
parameters = {'n_neighbors': [3, 4, 5, 10], 
              'weights': ['uniform', 'distance'],
              'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'leaf_size' : [10, 20, 30, 50],
              'p':[1,2]
             }

# Run the grid search
grid_obj = GridSearchCV(knn, parameters,cv=10,scoring= 'accuracy', n_jobs= -1, verbose= 3)

grid_obj = grid_obj.fit(X_train, y_train)

# Set the knn to the best combination of parameters
knn = grid_obj.best_estimator_

# Train the model using the training sets 
knn.fit(X_train,y_train)
y_pred_knn_test = knn.predict(X_test)
y_pred_knn_train = knn.predict(X_train)



Fitting 10 folds for each of 256 candidates, totalling 2560 fits
Wall time: 7.85 s


In [27]:
# Calculating the accuracy
acc_knn_test =  metrics.accuracy_score(y_test, y_pred_knn_test)
acc_knn_train = metrics.accuracy_score(y_train, y_pred_knn_train)

print( 'Accuracy of KNN model training : ', acc_knn_train )
print( 'Accuracy of KNN model testing : ', acc_knn_test )

roc_knn_train =metrics.roc_auc_score(y_train,y_pred_knn_train )
roc_knn_test = metrics.roc_auc_score(y_test,y_pred_knn_test )


Accuracy of KNN model training :  1.0
Accuracy of KNN model testing :  0.956140350877193


In [28]:
Algo_tuning.append('KNeighborsClassifier')
Training_tuning.append(acc_knn_train)
Testing_tuning.append(acc_knn_test)

In [29]:
%%time
# Create a GaussianNB
nb = GaussianNB()

# Hyperparameter Optimization
parameters = [{
 'var_smoothing': np.logspace(0,-9, num=100)}
   
]

# Run the grid search
grid_obj = GridSearchCV(nb, parameters,verbose=1, cv=10, n_jobs=-1,scoring= 'accuracy')

grid_obj = grid_obj.fit(X_train, y_train)

# Set the svc to the best combination of parameters
nb = grid_obj.best_estimator_


# Train the model using the training sets 
nb.fit(X_train,y_train)
y_pred_nb_test = nb.predict(X_test)
y_pred_nb_train = nb.predict(X_train)





Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Wall time: 2.19 s


In [30]:
# Calculating the accuracy
acc_nb_test = metrics.accuracy_score(y_test, y_pred_nb_test) 
acc_nb_train = metrics.accuracy_score(y_train, y_pred_nb_train) 

print( 'Accuracy of GaussianNB model training : ', acc_nb_train )
print( 'Accuracy of GaussianNB model testing : ', acc_nb_test )

roc_nb_train =metrics.roc_auc_score(y_train,y_pred_nb_train )
roc_nb_test = metrics.roc_auc_score(y_test,y_pred_nb_test )

Accuracy of GaussianNB model training :  0.9120879120879121
Accuracy of GaussianNB model testing :  0.9385964912280702


In [31]:
Algo_tuning.append('GaussianNB')
Training_tuning.append(acc_nb_train)
Testing_tuning.append(acc_nb_test)

In [32]:
%%time
DTT = DecisionTreeClassifier()
# Hyperparameter Optimization
parameters = {'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10, 50], 
              'min_samples_split': [2, 3, 50, 100],
              'min_samples_leaf': [1, 5, 8, 10],
              'splitter':['best',"random"],
            'random_state':[10,51,42,101]
             }

# Run the grid search
grid_obj = GridSearchCV(DTT, parameters,cv=10,scoring= 'accuracy', n_jobs= -1, verbose=3)

grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
DTT = grid_obj.best_estimator_

# Train the model using the training sets 
DTT.fit(X_train,y_train)
y_pred_DTT_test = DTT.predict(X_test)
y_pred_DTT_train = DTT.predict(X_train)





Fitting 10 folds for each of 3840 candidates, totalling 38400 fits
Wall time: 1min 24s


In [33]:
# Calculating the accuracy
acc_DTT_test = metrics.accuracy_score(y_test, y_pred_DTT_test)
acc_DTT_train = metrics.accuracy_score(y_train, y_pred_DTT_train) 

print( 'Accuracy of DecisionTreeClassifier model training : ', acc_DTT_train )
print( 'Accuracy of DecisionTreeClassifier model testing : ', acc_DTT_test )

roc_DTT_train =metrics.roc_auc_score(y_train,y_pred_DTT_train )
roc_DTT_test = metrics.roc_auc_score(y_test,y_pred_DTT_test )

Accuracy of DecisionTreeClassifier model training :  1.0
Accuracy of DecisionTreeClassifier model testing :  0.9035087719298246


In [34]:
Algo_tuning.append('DecisionTreeClassifier')
Training_tuning.append(acc_DTT_train)
Testing_tuning.append(acc_DTT_test)

In [35]:
%%time
# Random Forest Classifier

# Import library of RandomForestClassifier model
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest Classifier
rf = RandomForestClassifier()

# Hyperparameter Optimization
parameters = {'n_estimators': [4, 6, 9, 10, 15], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1, 5, 8],
               }

# Run the grid search
grid_obj = GridSearchCV(rf, parameters)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the rf to the best combination of parameters
rf = grid_obj.best_estimator_



# Train the model using the training sets 
rf.fit(X_train,y_train)
y_pred_rf_test = rf.predict(X_test)
y_pred_rf_train = rf.predict(X_train)



Wall time: 3min 20s


In [36]:
# Calculating the accuracy
acc_rf_test =  metrics.accuracy_score(y_test, y_pred_rf_test) 
acc_rf_train =  metrics.accuracy_score(y_train, y_pred_rf_train) 

print( 'Accuracy of Random Forest Classifier model training : ', acc_rf_train )
print( 'Accuracy of Random Forest Classifier model testing : ', acc_rf_test )

roc_rf_train =metrics.roc_auc_score(y_train,y_pred_rf_train )
roc_rf_test = metrics.roc_auc_score(y_test,y_pred_rf_test )


Accuracy of Random Forest Classifier model training :  0.9626373626373627
Accuracy of Random Forest Classifier model testing :  0.9473684210526315


In [37]:
Algo_tuning.append('RandomForestClassifier')
Training_tuning.append(acc_rf_train)
Testing_tuning.append(acc_rf_test)

In [38]:
%%time
# Create a AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier()

# Hyperparameter Optimization
parameters = [{
'n_estimators': [4, 6, 9, 10, 15], 
'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
'algorithm': ['SAMME', 'SAMME.R']
}]
# Run the grid search
grid_obj = GridSearchCV(adb, parameters,verbose=1, scoring = 'accuracy',cv=10, n_jobs=-1)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the svc to the best combination of parameters
adb = grid_obj.best_estimator_


# Train the model using the training sets 
adb.fit(X_train,y_train)
y_pred_adb_test = adb.predict(X_test)
y_pred_adb_train = adb.predict(X_train)




Fitting 10 folds for each of 80 candidates, totalling 800 fits
Wall time: 6.89 s


In [39]:
# Calculating the accuracy
acc_adb_test =  metrics.accuracy_score(y_test, y_pred_adb_test) 
acc_adb_train =  metrics.accuracy_score(y_train, y_pred_adb_train) 

print( 'Accuracy of adb model training : ', acc_adb_train )
print( 'Accuracy of adb model testing : ', acc_adb_test )

roc_adb_train =metrics.roc_auc_score(y_train,y_pred_adb_train )
roc_adb_test = metrics.roc_auc_score(y_test,y_pred_adb_test )

Accuracy of adb model training :  0.9604395604395605
Accuracy of adb model testing :  0.9473684210526315


In [40]:
Algo_tuning.append('AdaBoostClassifier')
Training_tuning.append(acc_adb_train)
Testing_tuning.append(acc_adb_test)

In [41]:
%%time
    from sklearn.ensemble import ExtraTreesClassifier
    # Create a ExtraTreesClassifier
    etc = ExtraTreesClassifier()

    # Hyperparameter Optimization
    parameters = [{
    'n_estimators': [10,20,30,50,100,200,300]}]
    # Run the grid search
    grid_obj = GridSearchCV(etc, parameters,verbose=1, scoring = 'accuracy',cv=10, n_jobs=-1)
    grid_obj = grid_obj.fit(X_train, y_train)

    # Set the svc to the best combination of parameters
    etc = grid_obj.best_estimator_


    # Train the model using the training sets 
    etc.fit(X_train,y_train)
    y_pred_etc_test = etc.predict(X_test)
    y_pred_etc_train = etc.predict(X_train)
    # Calculating the accuracy
    acc_etc_test = metrics.accuracy_score(y_test, y_pred_etc_test)
    acc_etc_train =  metrics.accuracy_score(y_train, y_pred_etc_train)

    print( 'Accuracy of etc model training : ', acc_etc_train )
    print( 'Accuracy of etc model testing : ', acc_etc_test )

    roc_etc_train =metrics.roc_auc_score(y_train,y_pred_etc_train )
    roc_etc_test = metrics.roc_auc_score(y_test,y_pred_etc_test )
   





Fitting 10 folds for each of 7 candidates, totalling 70 fits
Accuracy of etc model training :  1.0
Accuracy of etc model testing :  0.9824561403508771
Wall time: 2.22 s


In [42]:
Algo_tuning.append('ExtraTreesClassifier')
Training_tuning.append(acc_etc_train)
Testing_tuning.append(acc_etc_test)

In [43]:
import sklearn.externals as extjoblib
import joblib

In [44]:
filename = 'ExtraTreesClassifier1.sav'
joblib.dump(etc, filename)

loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

0.9824561403508771


In [45]:
Tuning_Testing = {'Algorithm':Algo_tuning,'Accuracy':Testing_tuning}
Tuning_Training = {'Algorithm':Algo_tuning,'Accuracy':Training_tuning}

In [46]:
Tuning_Testing = pd.DataFrame(Tuning_Testing)
Tuning_Training = pd.DataFrame(Tuning_Training)


In [47]:
Tuning_Training.sort_values('Accuracy',ascending=False)

,Algorithm,Accuracy
2,KNeighborsClassifier,1.000000
4,DecisionTreeClassifier,1.000000
7,ExtraTreesClassifier,1.000000
5,RandomForestClassifier,0.962637
0,Logistic Regression,0.960440
6,AdaBoostClassifier,0.960440
1,SVC,0.953846
3,GaussianNB,0.912088


In [48]:
Tuning_Testing.sort_values('Accuracy',ascending=False)

,Algorithm,Accuracy
7,ExtraTreesClassifier,0.982456
0,Logistic Regression,0.964912
1,SVC,0.964912
2,KNeighborsClassifier,0.956140
5,RandomForestClassifier,0.947368
6,AdaBoostClassifier,0.947368
3,GaussianNB,0.938596
4,DecisionTreeClassifier,0.903509


In [50]:
%time
def XGBClassifier(X_train,X_test,y_train,y_test):
    
    # XGBoost classifier most required parameters
    params={
         "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
         "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
         "min_child_weight" : [ 1, 3, 5, 7 ],
         "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
         "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] 
        }
    # Randomized Search
    from xgboost import XGBClassifier
    xgb_classifier = XGBClassifier()
    from sklearn.model_selection import RandomizedSearchCV
    random_search = RandomizedSearchCV(xgb_classifier, param_distributions=params,cv=10, scoring= 'accuracy', n_jobs= -1, verbose= 3)
    random_search.fit(X_train, y_train)
    xgb_classifier = random_search.best_estimator_
    # Train the model using the training sets 
    xgb_classifier.fit(X_train,y_train)
    y_pred_xgb_classifier_test = xgb_classifier.predict(X_test)
    y_pred_xgb_classifier_train = xgb_classifier.predict(X_train)
    # Calculating the accuracy
    acc_xgb_classifier_test =metrics.accuracy_score(y_test, y_pred_xgb_classifier_test)
    acc_xgb_classifier_train =metrics.accuracy_score(y_train, y_pred_xgb_classifier_train)

    print( 'Accuracy of xgb_classifier model training : ', acc_xgb_classifier_train )
    print( 'Accuracy of xgb_classifier model testing : ', acc_xgb_classifier_test )

    roc_xgb_classifier_train =metrics.roc_auc_score(y_train,y_pred_xgb_classifier_train )
    roc_xgb_classifier_test = metrics.roc_auc_score(y_test,y_pred_xgb_classifier_test )


Wall time: 0 ns


In [51]:
XGBClassifier(X_train,X_test,y_train,y_test)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[14:47:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:47:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of xgb_classifier model training :  0.9802197802197802
Accuracy of xgb_classifier model testing :  0.9649122807017544


In [ ]:
# filename = 'XGB_model1.sav'
# joblib.dump(xgb_classifier, filename)

# loaded_model = joblib.load(filename)
# result = loaded_model.score(X_test, y_test)
# print(result)

In [ ]:
print(classification_report(y_true =y_test,y_pred =y_pred_xgb_classifier_test))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_true =y_test,y_pred =y_pred_xgb_classifier_test).ravel()
tn, fp, fn, tp

In [ ]:
sns.heatmap(confusion_matrix(y_true =y_test,y_pred =y_pred_xgb_classifier_test))

In [53]:
from sklearn.ensemble import GradientBoostingClassifier
# Create a GradientBoostingClassifier
GBC = GradientBoostingClassifier()

In [54]:

# Hyperparameter Optimization
parameters = [{
"loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    'n_estimators': [4, 6, 9, 10, 15], 
              'max_features': ['log2', 'sqrt','auto'], 
     'max_depth': [3,5,10,15],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4, 6, 8,10],
               'min_samples_split': [5,10,15,20],
}]

# Run the grid search
from sklearn.model_selection import RandomizedSearchCV
grid_obj = RandomizedSearchCV(GBC, param_distributions=parameters, scoring= 'accuracy', n_jobs= -1, verbose= 3)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the svc to the best combination of parameters
GBC = grid_obj.best_estimator_


# Train the model using the training sets 
GBC.fit(X_train,y_train)
y_pred_GBC_classifier_test = GBC.predict(X_test)
y_pred_GBC_classifier_train = GBC.predict(X_train)
# Calculating the accuracy
acc_GBC_classifier_test =  metrics.accuracy_score(y_test, y_pred_GBC_classifier_test) 
acc_GBC_classifier_train =  metrics.accuracy_score(y_train, y_pred_GBC_classifier_train)  

print( 'Accuracy of GBC model training : ', acc_GBC_classifier_train )
print( 'Accuracy of GBC model testing : ', acc_GBC_classifier_test )

roc_GBC_classifier_train =metrics.roc_auc_score(y_train,y_pred_GBC_classifier_train )
roc_GBC_classifier_test = metrics.roc_auc_score(y_test,y_pred_GBC_classifier_test )



Fitting 5 folds for each of 10 candidates, totalling 50 fits
Accuracy of GBC model training :  0.9538461538461539
Accuracy of GBC model testing :  0.956140350877193
